In [1]:
from src.lstm import *
from sklearn.preprocessing import MinMaxScaler
import pickle

import warnings
warnings.filterwarnings('ignore')

Baixando dados da yfinance

In [10]:
df = download_data()
df.tail()

[*********************100%***********************]  1 of 1 completed


Price,ds,y
Ticker,,^BVSP
Date,,
2025-02-12,2025-02-12,124380.000000
2025-02-13,2025-02-13,124850.000000
2025-02-14,2025-02-14,128219.000000
2025-02-17,2025-02-17,128869.000000
2025-02-18,2025-02-18,128531.710938


Iniciando teste de parametros

In [26]:
# Definir parâmetros
previsao_dias = 30  # Número de dias para previsão (ajuste conforme necessário)
epoch = 10  # Número de épocas para o treinamento do modelo LSTM

# Preparar dados de treino e teste
treino, teste, df_treino_mms, x_treino, y_treino, normalize = prepare_data(df, previsao_dias)

# Construir o modelo LSTM
modelo_v1 = build_lstm_model(input_shape=(x_treino.shape[1], 1), unit=50, drop=0.2)

# Treinar o modelo
modelo_v1 = train_lstm_model(modelo_v1, x_treino, y_treino, epoch)

# Preparar dados de teste
x_teste2 = prepare_test_data(df, treino, teste, previsao_dias, normalize)

# Gerar previsões e calcular o MAPE
y_real = teste['y'].values  # Valores reais para o cálculo do MAPE
previsoes = generate_forecast(modelo_v1, x_teste2, normalize, y_real)

# Gerar o gráfico com os dados históricos e previsões
generate_plot(df, previsoes, start_date='2025-02-11', forecast_start='2025-02-18')

Epoch 1/10
38/38 [==============================] - 7s 32ms/step - loss: 0.0515 - mean_squared_error: 0.0515
Epoch 2/10
38/38 [==============================] - 1s 32ms/step - loss: 0.0110 - mean_squared_error: 0.0110
Epoch 3/10
38/38 [==============================] - 1s 33ms/step - loss: 0.0090 - mean_squared_error: 0.0090
Epoch 4/10
38/38 [==============================] - 1s 31ms/step - loss: 0.0073 - mean_squared_error: 0.0073
Epoch 5/10
38/38 [==============================] - 1s 32ms/step - loss: 0.0078 - mean_squared_error: 0.0078
Epoch 6/10
38/38 [==============================] - 1s 33ms/step - loss: 0.0083 - mean_squared_error: 0.0083
Epoch 7/10
38/38 [==============================] - 1s 31ms/step - loss: 0.0075 - mean_squared_error: 0.0075
Epoch 8/10
38/38 [==============================] - 1s 33ms/step - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 9/10
38/38 [==============================] - 1s 33ms/step - loss: 0.0064 - mean_squared_error: 0.0064
Epoch 10/10
1/1 [==

In [30]:
# Definir parâmetros
previsao_dias = 45

# Preparar dados de treino e teste
treino, teste, df_treino_mms, x_treino, y_treino, normalize = prepare_data(df, previsao_dias)

# Construir o modelo LSTM
modelo_v2 = build_lstm_model(input_shape=(x_treino.shape[1], 1), unit=50, drop=0.3)

# Treinar o modelo
modelo_v2 = train_lstm_model(modelo_v2, x_treino, y_treino, epoch=25)

# Preparar dados de teste
x_teste2 = prepare_test_data(df, treino, teste, previsao_dias, normalize)

# Gerar previsões e calcular o MAPE
y_real = teste['y'].values  # Valores reais para o cálculo do MAPE
previsoes = generate_forecast(modelo_v2, x_teste2, normalize, y_real)

# Gerar o gráfico com os dados históricos e previsões
generate_plot(df, previsoes, start_date='2025-02-11', forecast_start='2025-02-18')

Epoch 1/25
38/38 [==============================] - 9s 45ms/step - loss: 0.0680 - mean_squared_error: 0.0680
Epoch 2/25
38/38 [==============================] - 2s 46ms/step - loss: 0.0152 - mean_squared_error: 0.0152
Epoch 3/25
38/38 [==============================] - 2s 52ms/step - loss: 0.0115 - mean_squared_error: 0.0115
Epoch 4/25
38/38 [==============================] - 2s 60ms/step - loss: 0.0118 - mean_squared_error: 0.0118
Epoch 5/25
38/38 [==============================] - 3s 69ms/step - loss: 0.0104 - mean_squared_error: 0.0104
Epoch 6/25
38/38 [==============================] - 3s 77ms/step - loss: 0.0101 - mean_squared_error: 0.0101
Epoch 7/25
38/38 [==============================] - 3s 72ms/step - loss: 0.0097 - mean_squared_error: 0.0097
Epoch 8/25
38/38 [==============================] - 2s 64ms/step - loss: 0.0080 - mean_squared_error: 0.0080
Epoch 9/25
38/38 [==============================] - 2s 60ms/step - loss: 0.0078 - mean_squared_error: 0.0078
Epoch 10/25
38/38 [

Salvando arquivos pickle

In [29]:
normalize.fit(df['y'].values.reshape(-1, 1))
with open('normalize.pkl', 'wb') as f:
    pickle.dump(normalize, f)

In [16]:
# Salvando o modelo
with open('model/lstm.pkl', 'wb') as file:
    pickle.dump(modelo_v1, file)

Carregando arquivo para validação

In [17]:
# Carregando o modelo
with open('model/lstm.pkl', 'rb') as file:
    modelo = pickle.load(file)

In [18]:
modelo.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 30, 50)            10400     
                                                                 
 dropout_8 (Dropout)         (None, 30, 50)            0         
                                                                 
 lstm_9 (LSTM)               (None, 30, 50)            20200     
                                                                 
 dropout_9 (Dropout)         (None, 30, 50)            0         
                                                                 
 lstm_10 (LSTM)              (None, 30, 50)            20200     
                                                                 
 dropout_10 (Dropout)        (None, 30, 50)            0         
                                                                 
 lstm_11 (LSTM)              (None, 50)               